# Issue [#18](https://github.com/ai-cfia/llamaindex-db/issues/18)


In [1]:
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core import Settings
import os
from dotenv import load_dotenv
import psycopg
from pprint import pprint
import pickle

load_dotenv()

True

In [2]:
def save_to_pickle(data, filename):
    with open(filename, "wb") as file:
        pickle.dump(data, file)


def load_from_pickle(filename):
    with open(filename, "rb") as file:
        return pickle.load(file)

## Setup LLM and Embed Model


In [3]:
llm = AzureOpenAI(
    model="gpt-4",
    deployment_name="ailab-llm",
    api_key=os.getenv("API_KEY"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("API_VERSION"),
)

embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name="ada",
    api_key=os.getenv("API_KEY"),
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("API_VERSION"),
)

Settings.llm = llm
Settings.embed_model = embed_model

## Variables


In [3]:
database = os.getenv("DB_NAME")
host = os.getenv("DB_HOST")
password = os.getenv("DB_PASSWORD")
port = os.getenv("DB_PORT")
user = os.getenv("DB_USER")
llamaindex_db = "llamaindex_db_legacy"
llamaindex_schema = "v_0_0_1"

## Observed problem


### Find urls alternates

In [19]:
conn_string = (
    f"dbname={database} "
    f"user={user} "
    f"password={password} "
    f"host={host} "
    f"port={port}"
)
query = """
    SELECT c1.url, c2.url
    FROM louis_v005.crawl AS c1
    JOIN louis_v005.crawl AS c2
    ON (
        REGEXP_MATCH(c1.url, '.*/([a-z]{3})/([0-9]+)/([0-9]+)$') = 
        REGEXP_MATCH(c2.url, '.*/([a-z]{3})/([0-9]+)/([0-9]+)$')
        AND c1.url < c2.url
    );
    """
with psycopg.connect(conn_string) as conn:
    with conn.cursor() as cur:
        url_alternates = cur.execute(query).fetchall()
        
save_to_pickle(url_alternates, "url_alternates.pickle")


In [20]:
pprint(url_alternates[0:5])
print("len(url_alternates)", len(url_alternates))

[('https://inspection.canada.ca/about-cfia/media-relations/eng/1299073792503/1299076004509',
  'https://inspection.canada.ca/about-cfia/newsroom/eng/1299073792503/1299076004509'),
 ('https://inspection.canada.ca/about-cfia/media-relations/food-recall-warnings/eng/1299076382077/1299076493846',
  'https://inspection.canada.ca/about-cfia/newsroom/food-recall-warnings/eng/1299076382077/1299076493846'),
 ('https://inspection.canada.ca/about-cfia/acts-and-regulations/recent-regulatory-initiatives-and-notices-of-inten/eng/1299849033508/1299849093611',
  'https://inspection.canada.ca/about-cfia/acts-and-regulations/regulatory-initiatives-notices-of-intent/eng/1299849033508/1299849093611'),
 ('https://inspection.canada.ca/about-cfia/media-relations/stay-connected/eng/1299856061207/1299856119191',
  'https://inspection.canada.ca/about-cfia/newsroom/stay-connected/eng/1299856061207/1299856119191'),
 ('https://inspection.canada.ca/animal-health/humane-transport/eng/1300460032193/1300460096845',
  

### Urls that don't follow the same pattern

In [21]:
conn_string = (
    f"dbname={database} "
    f"user={user} "
    f"password={password} "
    f"host={host} "
    f"port={port}"
)
query = """
    SELECT c.url
    FROM louis_v005.crawl AS c
    WHERE NOT c.url ~ '.*/([a-z]{3})/([0-9]+)/([0-9]+)$';
    """
with psycopg.connect(conn_string) as conn:
    with conn.cursor() as cur:
        fringe_urls = cur.execute(query).fetchall()
        
save_to_pickle(fringe_urls, "fringe_urls.pickle")

In [22]:
pprint(fringe_urls)

[('https://inspection.canada.ca/about-cfia/transparency/corporate-management-reporting/reports-to-parliament/financial-reporting/quarter-ended-december-31-2019/eng//1582828285258/1582828285769',),
 ('https://inspection.canada.ca/preventive-controls/cleaning-and-sanitation-program/eng//1511374381399/1528206247934',),
 ('https://inspection.canada.ca/food-safety-for-industry/archived-food-guidance/meat-and-poultry-products/manual-of-procedures/directives-2017/2017-46/eng//1501247449418/1501247566311',),
 ('https://inspection.canada.ca/about-cfia/contact-a-cfia-office-by-telephone/eng//1313255382836/1313256130232',),
 ('https://inspection.canada.ca/food-safety-for-industry/toolkit-for-food-businesses/food-notices-for-industry/eng//1632510003942/1632510004676',)]


### Consequences

- Embeddings are generated in doubles for the same page.
- Duplicated search results
- Slightly degraded search accuracy as tested by our `api-test` tool

## Fix

- create a new view `unique_documents` similar to the `documents` view but built using unique entries in the `crawl` table, with a new field `url_id` (Ex: `/eng/1299073792503/1299076004509`)

The sql code for this would look like:

```sql
CREATE OR REPLACE VIEW louis_v005.unique_documents AS
WITH extracted_ids AS (
    SELECT
        id,
        url,
        substring(url, '/[a-z]{3}/[0-9]+/[0-9]+$') AS url_id,
        lang,
        title,
        md5hash,
        last_updated
    FROM louis_v005.crawl
),
unique_crawls AS (
    SELECT DISTINCT ON (url_id)
        id,
        url,
        url_id,
        lang,
        title,
        md5hash,
        last_updated
    FROM extracted_ids
)
SELECT 
    crawl.id,
    chunk.id AS chunk_id,
    crawl.url,
    crawl.url_id,
    crawl.lang,
    html_content.content AS html_content,
    crawl.title,
    chunk.title AS subtitle,
    chunk.text_content AS content,
    embedding.embedding,
    cardinality(token.tokens) AS tokens_count,
    crawl.last_updated,
    scoring.score
FROM 
    unique_crawls AS crawl
JOIN louis_v005.html_content ON crawl.md5hash = html_content.md5hash
JOIN louis_v005.html_content_to_chunk ON html_content.md5hash = html_content_to_chunk.md5hash
JOIN louis_v005.chunk ON html_content_to_chunk.chunk_id = chunk.id
JOIN louis_v005.token ON chunk.id = token.chunk_id
JOIN louis_v005.ada_002 embedding ON token.id = embedding.token_id
JOIN louis_v005.scoring ON crawl.id = scoring.entity_id;
```


In [11]:
conn_string = (
    f"dbname={database} "
    f"user={user} "
    f"password={password} "
    f"host={host} "
    f"port={port}"
)

create_view_query = """
CREATE OR REPLACE VIEW louis_v005.unique_documents AS
WITH extracted_ids AS (
    SELECT
        id,
        url,
        substring(url, '/[a-z]{3}/[0-9]+/[0-9]+$') AS url_id,
        lang,
        title,
        md5hash,
        last_updated
    FROM louis_v005.crawl
),
unique_crawls AS (
    SELECT DISTINCT ON (url_id)
        id,
        url,
        url_id,
        lang,
        title,
        md5hash,
        last_updated
    FROM extracted_ids
)
SELECT 
    crawl.id,
    chunk.id AS chunk_id,
    crawl.url,
    crawl.url_id,
    crawl.lang,
    html_content.content AS html_content,
    crawl.title,
    chunk.title AS subtitle,
    chunk.text_content AS content,
    embedding.embedding,
    cardinality(token.tokens) AS tokens_count,
    crawl.last_updated,
    scoring.score
FROM 
    unique_crawls AS crawl
JOIN louis_v005.html_content ON crawl.md5hash = html_content.md5hash
JOIN louis_v005.html_content_to_chunk ON html_content.md5hash = html_content_to_chunk.md5hash
JOIN louis_v005.chunk ON html_content_to_chunk.chunk_id = chunk.id
JOIN louis_v005.token ON chunk.id = token.chunk_id
JOIN louis_v005.ada_002 embedding ON token.id = embedding.token_id
JOIN louis_v005.scoring ON crawl.id = scoring.entity_id;
"""

check_duplicates_query = """
SELECT url_id, COUNT(DISTINCT url) AS unique_url_count
FROM louis_v005.unique_documents
GROUP BY url_id
HAVING COUNT(DISTINCT url) > 1;
"""


with psycopg.connect(conn_string) as conn:
    with conn.cursor() as cur:
        cur.execute(create_view_query)
        duplicates = cur.execute(check_duplicates_query).fetchall()
        if duplicates:
            print("Found duplicate url_id values:", duplicates)
            conn.rollback()
        else:
            print("No duplicates found. View created successfully.")
            conn.commit()



No duplicates found. View created successfully.
